In [1]:
# create dataset
import pandas as pd
base_dir = 'data/McGill-Billboard'
data_index = 'billboard-2.0-manychords.csv'

df_songs = pd.read_csv(f'{base_dir}/{data_index}')
df_songs.set_index('id', inplace=True)
len(df_songs)

719

In [2]:
df_songs.head(n=3)

,title,artist,no_chord_percent
id,,,
3,I Don't Mind,James Brown,0.049747
4,You've Got A Friend,"Roberta Flack,Donny Hathaway",0.050770
6,The Rose,Bette Midler,0.117244


In [3]:
test_ids = [1289, 736, 637, 270, 18] # songs to exclude for testing
df_dataset = df_songs.drop(index=test_ids)
len(df_dataset)

714

#### Splitting

In [4]:
import numpy as np
_SEED = 0

df_idxs = np.array(df_dataset.index.values)
rng = np.random.default_rng(_SEED)
rng.shuffle(df_idxs)

df_idxs[:10]

array([1167,    6,  986,  227,  743,  568,  107,  181,   27,  793])

In [9]:
import dataloader
from dataloader import ChromaSequenceDataset
from importlib import reload
reload(dataloader)

_LABEL_TYPE = 'majmin'
_SEQ_LEN = 64

pre_computed_seq = f'data/chordseq/{_LABEL_TYPE}_{_SEQ_LEN}.pkl'
ds = ChromaSequenceDataset(pre_computed_sequence=pre_computed_seq)

Loaded sequence data.


In [10]:
for train_split, val_split in ds.get_next_cv_split(df_idxs):
    print(train_split.shape, val_split.shape)

((43541, 64, 24), (43541, 64)) ((6882, 64, 24), (6882, 64))
((43575, 64, 24), (43575, 64)) ((6848, 64, 24), (6848, 64))
((43448, 64, 24), (43448, 64)) ((6975, 64, 24), (6975, 64))
((43485, 64, 24), (43485, 64)) ((6938, 64, 24), (6938, 64))
((42843, 64, 24), (42843, 64)) ((7580, 64, 24), (7580, 64))


### Training loop

#### Colab prep

In [ ]:
!pip install hyperopt
!pip install guildai # restart after install

In [ ]:
# colab prep
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/colab-handover/autochord/* ./
!ls

In [ ]:
# from dataloader import SimpleChromaDataset

# ds = SimpleChromaDataset(feat_label_files=('01_all_chroma_vectors.npy',
#                                            '01_all_chord_labels.npy'))

<hr style="border:1px solid gray">

In [14]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_addons as tfa

def K_plot_loss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [17]:
crf = tfa.layers.CRF(units=4)

In [18]:
crf.losses

[]

In [17]:
from dataloader import _CHROMA_FEAT_NAMES, _MAJMIN_CLASSES

_SEED = 0
_EPOCHS = 2 #10
_BATCH_SIZE = 2 #512
_SEQ_LEN = 64
_CKPT_PATH = 'models/chroma-seq-bilstm-crf-{cv}'

def init_bilstm_crf_model(base_linear_units=256, dropout=0.6, opt='adam', lr=0.01):
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(_SEQ_LEN, len(_CHROMA_FEAT_NAMES),)),
        layers.Bidirectional(
            layers.LSTM(units=base_linear_units, dropout=dropout,
                        stateful=False, return_sequences=False),
            merge_mode='concat'
        ),
#         layers.TimeDistributed(
#             layers.Dense(units=len(_MAJMIN_CLASSES), activation='softmax')
#         )
        tfa.layers.CRF(units=len(_MAJMIN_CLASSES))
    ])

    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

def sample_train_loop(ds):
    # cross validation loop
    tf.random.set_seed(_SEED)
    for cv_ix, (train, val) in enumerate(ds.get_next_cv_split()):
        print(f'----------- CV{cv_ix+1} -----------')
        train_dataset = tf.data.Dataset.from_tensor_slices((train.feats, train.labels)) \
                                       .take(100) \
                                       .shuffle(buffer_size=len(train), seed=_SEED, reshuffle_each_iteration=True) \
                                       .batch(_BATCH_SIZE)
        val_dataset = tf.data.Dataset.from_tensor_slices((val.feats, val.labels)) \
                                     .take(100) \
                                     .shuffle(buffer_size=len(val), seed=_SEED, reshuffle_each_iteration=True) \
                                     .batch(_BATCH_SIZE)

        print(f'Num train: {len(train)}, Num val: {len(val)}')
        assert(train.feats.shape[-1] == val.feats.shape[-1])
        print(f'Input features: {train.feats.shape[-1]}, Num classes: {ds.n_class}')
        
        model = init_simple_model()
        history = model.fit(train_dataset, validation_data=val_dataset, epochs=_EPOCHS)

        # get F1 score
        pred_chord_ixs = np.argmax(model.predict(val.feats, batch_size=_BATCH_SIZE), axis=1)
        print(f"val_fscore: {f1_score(y_true=val.labels, y_pred=pred_chord_ixs, average='macro')}")
        K_plot_loss(history)

        model.save(_CKPT_PATH.format(cv=cv_ix))

        break

sample_train_loop(ds)

**Manual tracking**

1. 256, 0.6, `Adam (0.01)` - `val loss: 3.2, val acc: 12.98%`
2. 1024, 0.6, `Adam (0.001)` - 

#### Automated tuning & tracking

In [ ]:
# colab
!cp -R /content/drive/MyDrive/colab-handover/autochord/guild-env-colab ./

In [9]:
from IPython.display import display

GUILD_HOME = 'guild-env/simple-chroma' # "guild-env-colab/simple-chroma"
DELETE_RUNS_ON_INIT = False
import guild.ipy as guild
guild.set_guild_home(GUILD_HOME)

if DELETE_RUNS_ON_INIT:
    deleted = guild.runs().delete(permanent=True)
    print("Deleted %i run(s)" % len(deleted))
else:
    display(guild.runs().head())

,run,operation,started,status,label
0,ff8c9ad6,hpset_trainloop(),2021-05-23 05:53:50,completed,bs=512 dp=0.6 hd=256 lr=0.001 opt=adam si=4
1,5d15565c,hpset_trainloop(),2021-05-23 05:52:48,completed,bs=512 dp=0.6 hd=256 lr=0.001 opt=adam si=3
2,856f7a7f,hpset_trainloop(),2021-05-23 05:51:46,completed,bs=512 dp=0.6 hd=256 lr=0.001 opt=adam si=2
3,66da07ec,hpset_trainloop(),2021-05-23 05:50:44,completed,bs=512 dp=0.6 hd=256 lr=0.001 opt=adam si=1
4,6f21924d,hpset_trainloop(),2021-05-23 05:49:43,completed,bs=512 dp=0.6 hd=256 lr=0.001 opt=adam si=0


In [29]:
_EPOCHS = 5
_TRAIN = None
_VAL = None

# function for guild tracking
def hpset_trainloop(hd=256, dp=0.6, opt='adam', lr=0.001, bs=512, si=0):
    '''
    Train loop with a specific set of hyperparams
    
    hd: hidden dim base size
    dp: dropout rate
    opt: optimizer, lr: learning rate
    bs: batch size
    si: CV split index
    '''
    tf.random.set_seed(_SEED)
    train = _TRAIN
    val = _VAL
    if (not train) or (not val):
        raise Exception("Missing data!")
    
    train_dataset = tf.data.Dataset.from_tensor_slices((train.feats, train.labels)) \
                                   .take(100) \
                                   .shuffle(buffer_size=len(train), seed=_SEED, reshuffle_each_iteration=True) \
                                   .batch(bs)
    val_dataset = tf.data.Dataset.from_tensor_slices((val.feats, val.labels)) \
                                 .take(100) \
                                 .shuffle(buffer_size=len(val), seed=_SEED, reshuffle_each_iteration=True) \
                                 .batch(bs)

    assert(train.feats.shape[-1] == val.feats.shape[-1])

    model = init_simple_model(base_linear_units=hd, dropout=dp, opt=opt, lr=lr)
    history = model.fit(train_dataset, validation_data=val_dataset, epochs=_EPOCHS, verbose=0)
    
    best_epoch = np.argmax(history.history['val_accuracy'])
    best_acc = history.history['val_accuracy'][best_epoch]
    
    # output metrics
    print(f"BE: {best_epoch+1}")
    print(f"VA: {best_acc}")
    
    return best_acc

In [ ]:
# tuning loop
from hyperopt import hp, tpe, fmin

def tuning_loop(hparams):
    global _TRAIN
    global _VAL
    
    print(hparams)

    avg_acc = 0.0
    num_runs = 0
    for cv_ix, (train, val) in enumerate(ds.get_next_cv_split()):
        _TRAIN = train
        _VAL = val
        run, acc = guild.run(hpset_trainloop,
                             hd=int(hparams['base_hidden_dim']),
                             dp=hparams['drop_rate'],
                           opt=hparams['opt'],
                             lr=hparams['lr'], 
                             bs=int(hparams['batch_size']),
                             si=cv_ix)
        
        num_runs += 1
        # if hyperparams fail miserably on one split,
        # no need to check other splits
        if acc < 0.5:
            return 1.0
            
        avg_acc += acc
    
    avg_acc /= num_runs
    return (1-avg_acc) # since we're using fmin

hparams = {
    'base_hidden_dim': hp.choice('base_hidden_dim', [256, 512, 1024]),
    'drop_rate': hp.choice('drop_rate', [0.9, 0.7, 0.5, 0.3]),
    'opt': hp.choice('opt', ['adam']),
    'lr': hp.choice('lr', [1e-2, 1e-3, 3e-4, 1e-4]),
    'batch_size': hp.choice('batch_size', [16, 32, 64]),
}

best = fmin(tuning_loop, hparams, algo=tpe.suggest, max_evals=1)
print(best)

In [ ]:
# colab
!cp -R guild-env-colab /content/drive/MyDrive/colab-handover/autochord/

In [18]:
runs = guild.runs()
df_exps = runs.compare()

_COMPARE_COLS = ['bs','dp','hd','lr','opt','si','BE','VA']
comps = df_exps[_COMPARE_COLS][:65]
comps[comps.VA > 0.52]

,bs,dp,hd,lr,opt,si,BE,VA
9,256,0.6,256,0.0003,adam,0,2.0,0.52030
12,512,0.3,256,0.0003,adam,2,2.0,0.52078
17,512,0.3,512,0.0003,adam,2,1.0,0.52138
36,256,0.5,256,0.0001,adam,3,2.0,0.52239
37,256,0.5,256,0.0001,adam,2,1.0,0.52428
39,256,0.5,256,0.0001,adam,0,2.0,0.52686
51,256,0.5,256,0.0003,adam,0,1.0,0.52050
53,256,0.5,256,0.0001,adam,0,2.0,0.52666


#### Scratch

In [ ]:
import numpy as np

# try limiting class
ds.chroma_vectors = np.load('01_all_chroma_vectors.npy')
ds.chord_labels = np.load('01_all_chord_labels.npy')
filt = np.isin(ds.chord_labels, (0,1))
ds.chroma_vectors = ds.chroma_vectors[filt]
ds.chord_labels = ds.chord_labels[filt]
ds.classes = set(ds.chord_labels)
ds.n_class = len(ds.classes)
print('Loaded features and labels.')
ds.train_split, ds.val_splits, ds.test_split = ds.get_splits()
print('Split into train, val, test.')
print(ds.n_class)

In [10]:
print(ds.chroma_vectors[0], ds.chord_labels[0])
print(ds.chroma_vectors[-113], ds.chord_labels[-113])

[0.198482  0.        0.        0.635556  0.741292  1.0043    0.81444
 0.0292819 0.141189  0.80793   0.91015   0.823603  1.22066   0.0969013
 0.197437  0.860228  1.16515   1.13561   0.42842   0.112475  1.49297
 0.556156  0.562561  0.864485 ] 0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 17


In [33]:
filt = [ds.chord_labels == 0]
np.mean(ds.chroma_vectors[filt], axis=0)

/home/cjbayron/virtualenvs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


array([0.32049295, 0.24610202, 0.28693406, 0.25687285, 0.20151652,
       0.29747371, 0.20591433, 0.27151966, 0.22509008, 0.21612551,
       0.26522816, 0.21338926, 0.7405679 , 0.48997662, 0.60600057,
       0.60080249, 0.48543003, 0.73010563, 0.44549839, 0.73231232,
       0.53258909, 0.54258336, 0.6778894 , 0.48055285])

In [34]:
filt = [ds.chord_labels == 1]
np.mean(ds.chroma_vectors[filt], axis=0)

/home/cjbayron/virtualenvs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


array([0.20761943, 0.14172344, 0.25146699, 1.23340365, 0.1735976 ,
       0.17857491, 0.14191071, 0.30587265, 0.20928596, 0.19817245,
       0.54753261, 0.16978452, 0.48747069, 0.36887956, 0.55286642,
       2.05746208, 0.21877638, 0.64896599, 0.25239646, 1.45968816,
       0.40032339, 0.2984598 , 1.69132596, 0.2568249 ])

In [21]:
filt = [np.sum(ds.chroma_vectors, axis=1) == 0]

In [24]:
from collections import Counter
Counter(ds.chord_labels[filt])

/home/cjbayron/virtualenvs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


Counter({0: 49874,
         2: 203,
         8: 537,
         22: 150,
         13: 51,
         6: 464,
         7: 249,
         12: 330,
         9: 192,
         4: 141,
         24: 34,
         1: 399,
         17: 97,
         15: 244,
         10: 434,
         5: 547,
         14: 77,
         3: 414,
         11: 269,
         20: 80,
         18: 140,
         23: 21,
         16: 141,
         19: 29,
         21: 1})

In [11]:
history.history['loss'][-1]

0.03958175703883171

In [6]:
# test loading
m = tf.keras.models.load_model(_CKPT_PATH.format(cv=0))
m.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               6400      
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              263168    
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_7 (Dense)              (None, 25)                6425      
Total params: 538,393
Trainable params: 538,393
Non-trainable params: 0
________________________________________________

In [ ]:
# add callbacks
# add tuning
# mind the seeding!